In [ ]:
# Solution to custom environment

In [ ]:
from myenv import ENVIRONMENT
import numpy as np
import torch
import cv2
import random

In [ ]:
HM_EPISODES = 25000
MOVE_PENALTY = 1
ENEMY_PENALTY = 300
FOOD_REWARD = 25
epsilon = 0.9
EPS_DECAY = 0.9998  
SHOW_EVERY = 1000  
DISPLAY_EVERY= 500
SIZE = 10
LEARNING_RATE = 0.1
DISCOUNT = 0.95
total_reward = 0

In [ ]:
def get_q_table(start_q_table=None,size=10,action=4):
    
    if start_q_table is None:
        q_table = np.random.randn(size,size,action)
        print(q_table.size)

    else:
        with open(start_q_table, "rb") as f:
            q_table = pickle.load(f)
    
    return q_table


In [ ]:
env = ENVIRONMENT(diagonal=False,size=10,num_enemy = 3, num_food = 1)
q = get_q_table(size=10)
# Test Environmet by rendering once

print(env.startover())

for i in range(100):
    print(env.step(np.random.randint(0,4)))
    env.render()
    cv2.waitKey(100)
cv2.destroyAllWindows()

print(env.startover())


In [ ]:
# for i in range(1):
#     print(env.step(2))
#     env.render()
# cv2.waitKey(0)    
# cv2.destroyAllWindows()


In [ ]:
# Improve q-value lookup table
for episode in range(HM_EPISODES):
    
    state, reward, done = env.startover(newpos=True)
    
    while not done:
    
        current_q = q[state[0],state[1],:]
        
        if random.random() > epsilon:
            
            action = np.argmax(current_q)

        else:
            action = np.random.randint(0,4)

        next_state, (next_reward, done) = env.step(action)
        total_reward += next_reward
        future_q = q[next_state[0],next_state[1],:]
        q[state[0],state[1],action] = (1 - LEARNING_RATE) * current_q[action] + LEARNING_RATE * ( next_reward + DISCOUNT * max(future_q) - current_q[action])

        if done and next_reward == 100:
            q[state[0],state[1] :] = 0

#         if done and next_reward == -100:
#             print('Hell i fucked!')
            
        if episode%SHOW_EVERY == 0:
            env.render()
            cv2.waitKey(100)
            
        state = next_state
            
    cv2.destroyAllWindows()
    epsilon *= EPS_DECAY
    
    if episode%DISPLAY_EVERY == 0:
        print('Episode: ',episode,'state:',state,'| Total Average Reward:', total_reward/500,'| Epsilon:', epsilon)
        total_reward= 0

In [ ]:
## TESTING
# import time
# with open(f"qtable-{int(time.time())}.pickle", "wb") as f:
#     pickle.dump(q_table, f)


In [ ]:
# from myenv import ENVIRONMENT
# import cv2 
# env = ENVIRONMENT(num_enemy = 3, size= 20)


In [ ]:
# env.render()
# cv2.waitKey(0)
# cv2.destroyAllWindows()
